First we will begin by downloading the required libraries and download the kaggle dataset  

In [1]:
import kagglehub
import pandas as pd
import os

# Download latest version
path = kagglehub.dataset_download("benroshan/factors-affecting-campus-placement")

DATA EXPLORATION

Step 1: Read the data into the df and inspect 

In [2]:
# this will actually open the dataframe in the enviroment 
csv_file = os.path.join(path, "Placement_Data_Full_Class.csv")  
df = pd.read_csv(csv_file)

In [3]:
df.head()

sl_no gender  ssc_p    ssc_b  hsc_p    hsc_b     hsc_s  degree_p  \
0      1      M  67.00   Others  91.00   Others  Commerce     58.00   
1      2      M  79.33  Central  78.33   Others   Science     77.48   
2      3      M  65.00  Central  68.00  Central      Arts     64.00   
3      4      M  56.00  Central  52.00  Central   Science     52.00   
4      5      M  85.80  Central  73.60  Central  Commerce     73.30   

    degree_t workex  etest_p specialisation  mba_p      status    salary  
0   Sci&Tech     No     55.0         Mkt&HR  58.80      Placed  270000.0  
1   Sci&Tech    Yes     86.5        Mkt&Fin  66.28      Placed  200000.0  
2  Comm&Mgmt     No     75.0        Mkt&Fin  57.80      Placed  250000.0  
3   Sci&Tech     No     66.0         Mkt&HR  59.43  Not Placed       NaN  
4  Comm&Mgmt     No     96.8        Mkt&Fin  55.50      Placed  425000.0

Step 2: Begin some cleaning process so we can inspect a bit more 

In [4]:

# this is the data cleaning step for the data visiluization, we will use one hot encoding for the ML model 

# The following functions are used for these conversions: 
def hsc_s_converter(x): 
    dic = {
        'Commerce': 0, 
        'Science': 1, 
        'Arts': 2
    }
    return dic[x]

def hsc_b_converter(x): 
    dic = {
        'Central': 0, 
        'Others': 1, 
    }
    return dic[x]

def degree_t_converter(x): 
    degree_dict = dict(enumerate(df.degree_t.unique()))
    inverted_degree_dict = {v: k for k, v in degree_dict.items()}
    inverted_degree_dict[x]

def specialisation_converter(x): 
    degree_dict = dict(enumerate(df.specialisation.unique()))
    inverted_degree_dict = {v: k for k, v in degree_dict.items()}
    inverted_degree_dict[x]


# Here we will apply the encodings 
df['gender_int'] = df['gender'].apply(lambda x: 1 if x=='M' else 0)  # M: 1 , F:0
df['status_int'] = df['status'].apply(lambda x: 1 if x=='Placed' else 0)  # Placed: 1 , NOT Placed:0
df['hsc_s_int'] = df.hsc_s.apply(lambda x: hsc_s_converter(x))
df['hsc_b_int'] = df.hsc_b.apply(lambda x: hsc_b_converter(x))
df['degree_t_int'] = df.degree_t.apply(lambda x: degree_t_converter(x))
df['workex_int'] = df.workex.apply(lambda x: 1 if x=='Yes' else 0)
df['specialisation_int'] = df.specialisation.apply(lambda x: specialisation_converter(x))

In [5]:
df.rename(columns={
    'sl_no': 'Serial_Number',
    'gender': 'Gender',
    'ssc_p': 'Secondary_School_Percentage',
    'ssc_b': 'Secondary_School_Board',
    'hsc_p': 'Higher_Secondary_Percentage',
    'hsc_b': 'Higher_Secondary_Board',
    'hsc_s': 'Higher_Secondary_Stream',
    'degree_p': 'Degree_Percentage',
    'degree_t': 'Degree_Type',
    'workex': 'Work_Experience',
    'etest_p': 'Employability_Test_Percentage',
    'specialisation': 'MBA_Specialisation',
    'mba_p': 'MBA_Percentage',
    'status': 'Placement_Status',
    'salary': 'Salary',
    # Encoded columns:
    'gender_int': 'Gender_Code',
    'status_int': 'Placement_Code',
    'hsc_s_int': 'Higher_Secondary_Stream_Code',
    'hsc_b_int': 'Higher_Secondary_Board_Code',
    'degree_t_int': 'Degree_Type_Code',
    'workex_int': 'Work_Experience_Code',
    'specialisation_int': 'MBA_Specialisation_Code'
}, inplace=True)


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.heatmap(df.corr(numeric_only=True))

<Axes: >

In [7]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='Secondary_School_Percentage', y='Higher_Secondary_Percentage', hue='Placement_Code', data=df)
sns.regplot(x='Secondary_School_Percentage', y='Higher_Secondary_Percentage', color='black', scatter=False, data=df)

plt.xlabel("Secondary_School_Percentage")
plt.ylabel("Higher_Secondary_Percentage")
plt.title("Students who do better in Higher also do better in Lower school, these students also get jobs")


Text(0.5, 1.0, 'Students who do better in Higher also do better in Lower school, these students also get jobs')

In [8]:


import scipy.stats as stats
import numpy as np

dfPlot = df.groupby(['MBA_Specialisation', 'Degree_Type']).agg(
    Placement_Mean=('Placement_Code', 'mean'),
    Placement_Count=('Placement_Code', 'count'),
    Salary_Mean=('Salary', 'mean')
    ).reset_index()

dfPlot = dfPlot.sort_values(by=['Placement_Mean'], ascending=False).reset_index(drop=True)
dfPlot['Placement_Mean'] = dfPlot['Placement_Mean'].apply(lambda x: x*100)
dfPlot['Combined_Degree'] = dfPlot[['Degree_Type','MBA_Specialisation']].apply(lambda x: f'{x[0]} & {x[1]}',axis=1)
fig, ax1 = plt.subplots(figsize=(10, 6))
sns.barplot(data=dfPlot, x='Combined_Degree', y='Placement_Mean', ax=ax1)
plt.xlabel('Percent Placed')
plt.title("Placemnt Percentage Based On Degree Combo")
plt.xticks(rotation=60)  # Makes labels vertical

ax2 = ax1.twinx()

ax2.plot(dfPlot.Salary_Mean, label='Avg Salary', color='red')
ax2.set_ylabel("AVG Salary")


# dfPlot

/var/folders/_k/mghhvbbn09b2dr27609xjygc0000gn/T/ipykernel_39930/562587451.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfPlot['Combined_Degree'] = dfPlot[['Degree_Type','MBA_Specialisation']].apply(lambda x: f'{x[0]} & {x[1]}',axis=1)


Text(0, 0.5, 'AVG Salary')

In [9]:
dfPlot['Placemnt_To_Sal'] = dfPlot[['Placement_Mean','Salary_Mean']].apply(lambda x: (x[0]*x[1])/100000,axis=1)

dfPlot = dfPlot.sort_values(by='Placemnt_To_Sal', ascending=False)
sns.barplot(data=dfPlot, y=dfPlot.Combined_Degree, x=dfPlot.Placemnt_To_Sal)
plt.xlabel("Expected Salary (Thousands)")
plt.ylabel("")

plt.title("Combined Degree To Expected Salary")



/var/folders/_k/mghhvbbn09b2dr27609xjygc0000gn/T/ipykernel_39930/588491297.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfPlot['Placemnt_To_Sal'] = dfPlot[['Placement_Mean','Salary_Mean']].apply(lambda x: (x[0]*x[1])/100000,axis=1)


Text(0.5, 1.0, 'Combined Degree To Expected Salary')

MODEL BUILDING

In [10]:


# Here we pick the features we want to use in our ML model 
df_ml = df[['Gender', 'Secondary_School_Percentage',
       'Higher_Secondary_Percentage',
       'Higher_Secondary_Stream',
       'Degree_Percentage', 'Degree_Type', 'Work_Experience',
       'Employability_Test_Percentage', 'MBA_Specialisation', 'MBA_Percentage',
       'Placement_Status', 'Salary']]

# Inspect again which columns will require one hot encoding 
# df_ml.info()

# Specify those columns columns in a list which is then used to create the one hot encoding 
categorical_cols = ['Gender', 'Higher_Secondary_Stream', 'Degree_Type', 'Work_Experience', 'MBA_Specialisation']
df_ml['Placement_Status']=  df_ml['Placement_Status'].apply(lambda x: 0 if x == 'Not Placed' else 1)

# Here we create the one hot encoding and confirm that it worked (looks like it did)
df_ml = pd.get_dummies(df_ml, columns=categorical_cols)
df_ml.dtypes

/var/folders/_k/mghhvbbn09b2dr27609xjygc0000gn/T/ipykernel_39930/2483126274.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml['Placement_Status']=  df_ml['Placement_Status'].apply(lambda x: 0 if x == 'Not Placed' else 1)


Secondary_School_Percentage         float64
Higher_Secondary_Percentage         float64
Degree_Percentage                   float64
Employability_Test_Percentage       float64
MBA_Percentage                      float64
Placement_Status                      int64
Salary                              float64
Gender_F                               bool
Gender_M                               bool
Higher_Secondary_Stream_Arts           bool
Higher_Secondary_Stream_Commerce       bool
Higher_Secondary_Stream_Science        bool
Degree_Type_Comm&Mgmt                  bool
Degree_Type_Others                     bool
Degree_Type_Sci&Tech                   bool
Work_Experience_No                     bool
Work_Experience_Yes                    bool
MBA_Specialisation_Mkt&Fin             bool
MBA_Specialisation_Mkt&HR              bool
dtype: object

In [11]:
# Here we check for some potential Null values, this is important that we do 
# this here as our data set is relativly small and we want to keep as much data 
# as possible 
df_ml[['Placement_Status', 'Salary']].isna().sum()


Placement_Status     0
Salary              67
dtype: int64

In [12]:
# In order to preserve as many rows as possible we will split our DB into one for Salary and One for outcome 
# This will ensure we keep as much data as possibl e

# The positon outcome we will drop the salary column as this has NA values and is not apart of our model training


df_ml_PS = df_ml.drop(columns=["Salary", "Employability_Test_Percentage"])

# For our salary, we will drop all of NA values since we do want any NA values in our validation set
df_ml_salary = df_ml.drop(columns=["Employability_Test_Percentage"])
df_ml_salary = df_ml.dropna()


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

boolean_features = ['Gender_F','Gender_M','Degree_Type_Comm&Mgmt','Degree_Type_Others','Degree_Type_Sci&Tech','Work_Experience_No','Work_Experience_Yes','MBA_Specialisation_Mkt&Fin','MBA_Specialisation_Mkt&HR']
num_features = ['Secondary_School_Percentage', 'Higher_Secondary_Percentage',
       'Degree_Percentage', 'Employability_Test_Percentage', 'MBA_Percentage',
       'Placement_Status']
scaler = MinMaxScaler()
# Split and scale our data our data into training and testing

features = num_features + boolean_features

X = df_ml_salary[features]
y = df_ml_salary['Salary']

X_train_salary, X_test_salary, y_train_salary, y_test_salary = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_salary[num_features] = scaler.fit_transform(X_train_salary[num_features])
X_test_salary[num_features]  = scaler.transform(X_test_salary[num_features])




1. LINEAR REGRESSION MODEL FOR SALARY 

In [15]:
from sklearn.linear_model import LinearRegression
lr_model_salary = LinearRegression().fit(X_train_salary, y_train_salary) 

In [17]:
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

lr_model_predict_salary = lr_model_salary.predict(X_test_salary)
lr_model_predict_score = r2_score(y_test_salary, lr_model_predict_salary)

mse = mean_squared_error(y_test_salary, lr_model_predict_salary)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_salary, lr_model_predict_salary)
mdae = median_absolute_error(y_test_salary, lr_model_predict_salary)
r2 = r2_score(y_test_salary, lr_model_predict_salary)
print(f"""
MSE: {mse}
RMSE: {rmse}
MAE: {mae}
MDAE: {mdae}
R2: {r2}
""")



MSE: 9528801689.6
RMSE: 97615.5811825141
MAE: 65555.2
MDAE: 42952.0
R2: -0.12708432610885279



Our outcome here is very poor and have huge amounts of error, we will try to remove some potentially noisy features 

In [18]:
# We will now inspect the correlation and see which values may be best for prediction of salary 
sal_cor = df.corr(numeric_only=True).sort_values(by='Salary')['Salary']
print(sal_cor.index)
print(sal_cor)

Index(['Degree_Percentage', 'Higher_Secondary_Stream_Code',
       'Higher_Secondary_Board_Code', 'Secondary_School_Percentage',
       'Serial_Number', 'Higher_Secondary_Percentage', 'Work_Experience_Code',
       'Gender_Code', 'MBA_Percentage', 'Employability_Test_Percentage',
       'Salary', 'Placement_Code'],
      dtype='object')
Degree_Percentage               -0.019272
Higher_Secondary_Stream_Code    -0.017923
Higher_Secondary_Board_Code     -0.007549
Secondary_School_Percentage      0.035330
Serial_Number                    0.063764
Higher_Secondary_Percentage      0.076819
Work_Experience_Code             0.136920
Gender_Code                      0.158912
MBA_Percentage                   0.175013
Employability_Test_Percentage    0.178307
Salary                           1.000000
Placement_Code                        NaN
Name: Salary, dtype: float64


In [19]:
# first create the smaller df with the columns we care about
df_ml_salary_shortend = pd.DataFrame(df[['Degree_Type','Higher_Secondary_Percentage', 'Work_Experience',
       'Gender', 'MBA_Percentage', 'Employability_Test_Percentage', 'MBA_Specialisation','Salary']])

# now we will have to one hot encode these once again 
categorical_cols = ['Gender','Degree_Type', 'Work_Experience', 'MBA_Specialisation']

# Here we create the one hot encoding once again with the new DF
df_ml_salary_shortend = pd.get_dummies(df_ml_salary_shortend, columns=categorical_cols)


In [20]:
# check that it worked
df_ml_salary_shortend = df_ml_salary_shortend.dropna()
df_ml_salary_shortend.info()
print(df_ml_salary_shortend.shape)


<class 'pandas.core.frame.DataFrame'>
Index: 148 entries, 0 to 213
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Higher_Secondary_Percentage    148 non-null    float64
 1   MBA_Percentage                 148 non-null    float64
 2   Employability_Test_Percentage  148 non-null    float64
 3   Salary                         148 non-null    float64
 4   Gender_F                       148 non-null    bool   
 5   Gender_M                       148 non-null    bool   
 6   Degree_Type_Comm&Mgmt          148 non-null    bool   
 7   Degree_Type_Others             148 non-null    bool   
 8   Degree_Type_Sci&Tech           148 non-null    bool   
 9   Work_Experience_No             148 non-null    bool   
 10  Work_Experience_Yes            148 non-null    bool   
 11  MBA_Specialisation_Mkt&Fin     148 non-null    bool   
 12  MBA_Specialisation_Mkt&HR      148 non-null    bool   


In [21]:

scaler = MinMaxScaler()
# Split and scale our data our data into training and testing

num_features = [
    'Higher_Secondary_Percentage',
    'MBA_Percentage',
    'Employability_Test_Percentage'
]

boolean_features = [
    'Gender_F',
    'Gender_M',
    'Degree_Type_Comm&Mgmt',
    'Degree_Type_Others',
    'Degree_Type_Sci&Tech',
    'Work_Experience_No',
    'Work_Experience_Yes',
    'MBA_Specialisation_Mkt&Fin',
    'MBA_Specialisation_Mkt&HR'
]


features = num_features + boolean_features

X = df_ml_salary_shortend[features]
y = df_ml_salary_shortend['Salary']
X_train_salary, X_test_salary, y_train_salary, y_test_salary = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_salary[num_features] = scaler.fit_transform(X_train_salary[num_features])
X_test_salary[num_features]  = scaler.transform(X_test_salary[num_features])



In [22]:
from sklearn.linear_model import LinearRegression
lr_model_salary = LinearRegression().fit(X_train_salary, y_train_salary) 

In [23]:
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

lr_model_predict_salary = lr_model_salary.predict(X_test_salary)
lr_model_predict_score = r2_score(y_test_salary, lr_model_predict_salary)

mse = mean_squared_error(y_test_salary, lr_model_predict_salary)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_salary, lr_model_predict_salary)
mdae = median_absolute_error(y_test_salary, lr_model_predict_salary)
r2 = r2_score(y_test_salary, lr_model_predict_salary)
print(f"""
MSE: {mse}
RMSE: {rmse}
MAE: {mae}
MDAE: {mdae}
R2: {r2}
""")



MSE: 8485874710.188348
RMSE: 92118.80758123363
MAE: 60422.798755756005
MDAE: 44837.844772154494
R2: -0.0037249898500384404



In [24]:
# Old: 
# MSE: 9528801689.6
# RMSE: 97615.5811825141
# MAE: 65555.2
# MDAE: 42952.0
# R2: -0.12708432610885279


# New: 
# MSE: 8485874710.188348
# RMSE: 92118.80758123363
# MAE: 60422.798755756005
# MDAE: 44837.844772154494
# R2: -0.0037249898500384404

# we see here a bit of an imporvemnt with our scores however not much. This does however make sense since we see little correlation
# between salary and the other attributes within our data set 

2. Logistic Regression for Outcome

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

boolean_features = ['Gender_F','Gender_M','Degree_Type_Comm&Mgmt','Degree_Type_Others','Degree_Type_Sci&Tech','Work_Experience_No','Work_Experience_Yes','MBA_Specialisation_Mkt&Fin','MBA_Specialisation_Mkt&HR']
num_features = ['Secondary_School_Percentage', 'Higher_Secondary_Percentage',
       'Degree_Percentage','MBA_Percentage']
scaler = MinMaxScaler()
# Split and scale our data our data into training and testing

features = num_features + boolean_features

X = df_ml_PS[features]
y = df_ml_PS['Placement_Status']

X_train_PS, X_test_PS, y_train_PS, y_test_PS = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_PS[num_features] = scaler.fit_transform(X_train_PS[num_features])
X_test_PS[num_features]  = scaler.transform(X_test_PS[num_features])




In [54]:

# scaler = MinMaxScaler()

# # Split and and scale the data 
# X_train_PS, X_test_PS, y_train_PS, y_test_PS = train_test_split(df_ml_PS.drop(columns=['Placement_Status']), df_ml_PS['Placement_Status'], test_size=0.2, random_state=42, stratify=df_ml['Placement_Status'])
# X_train_scaled_PS, X_test_scaled_PS = scaler.fit_transform(X_train_PS), scaler.transform(X_test_PS)  


In [55]:
from sklearn.linear_model import LogisticRegression

lgr_classifer = LogisticRegression(max_iter=1000, random_state=42)
lgr_classifer.fit(X_train_PS, y_train_PS)


LogisticRegression(max_iter=1000, random_state=42)

In [56]:
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

y_pred_lgr = lgr_classifer.predict(X_test_PS)
print(classification_report(y_test_PS, y_pred_lgr))



              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.85      0.94      0.89        31

    accuracy                           0.84        43
   macro avg       0.82      0.76      0.78        43
weighted avg       0.83      0.84      0.83        43



3. RandomForest Classifer for job outcome outcome 

In [57]:
from sklearn.ensemble import RandomForestClassifier
rdmf_classifer = RandomForestClassifier(n_estimators=100, random_state=42)
rdmf_classifer.fit(X_train_PS, y_train_PS)


RandomForestClassifier(random_state=42)

In [58]:
y_pred_dmf = rdmf_classifer.predict(X_test_PS)
print(classification_report(y_test_PS, y_pred_lgr))


              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.85      0.94      0.89        31

    accuracy                           0.84        43
   macro avg       0.82      0.76      0.78        43
weighted avg       0.83      0.84      0.83        43



4. Neural Net Model (just for fun)

In [63]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(random_state=42, max_iter=1000).fit(X_train_PS, y_train_PS)


In [64]:
y_pred_mlp = mlp_clf.predict(X_test_PS)
print(classification_report(y_test_PS, y_pred_mlp))


              precision    recall  f1-score   support

           0       0.82      0.75      0.78        12
           1       0.91      0.94      0.92        31

    accuracy                           0.88        43
   macro avg       0.86      0.84      0.85        43
weighted avg       0.88      0.88      0.88        43

